In [52]:
import os
import urllib
import requests
import json
import csv
import pandas as pd
import re

data_list = []
header_list = ["Formal Name", "Location Address", "Suburb",
               "Postcode",  "Latitude and longitude"]
jw_list = []
# google map api
key = 

with open("../data/raw/hospital.csv", mode="r", encoding="utf-8-sig") as f:
    # based on the file, create csv reader
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        address = row[5] + ',' + row[6] + ',Australia'
        print(address)
        url = 'https://maps.googleapis.com/maps/api/geocode/json?{}&key={}'.format(
            urllib.parse.urlencode({'address': address}), key)
        res = requests.get(url)
        res.encoding = 'UTF-8'
        # return json
        json_data = json.loads(res.text)
        result = json_data['results']
        # extract location from json
        geo = result[0]['geometry']['location']
        longitude = geo['lng']
        latitude = geo['lat']
        jw_list = [row[2], row[5], row[6], row[7], str(latitude) + "," + str(longitude)]
        data_list.append(jw_list)
        print(latitude, longitude)

# open data with write mode, create a new line between two data
with open("../data/raw/hospital_new.csv", mode="w", encoding="utf-8-sig", newline="") as f:

    writer = csv.writer(f)

    # create header
    writer.writerow(header_list)

    # write the data
    writer.writerows(data_list)

os.system("pause")

31-33 Albert Road,South Melbourne,Australia
-37.8342645 144.9722408
69 Vermont Street,Wodonga,Australia
-36.1319554 146.8798654
Borella,Albury,Australia
-36.0757523 146.9308464
20 Cooper Street,Alexandra,Australia
-37.1940818 145.7172472
Commercial Road,Melbourne,Australia
-37.8458528 144.9863747
30 O'Donnell Avenue,Myrtleford,Australia
-36.5544039 146.7293605
Albert Street,Upper Ferntree Gully,Australia
-37.8989287 145.3132882
57 Garsed Street,Bendigo,Australia
-36.765123 144.2797671
145 Studley Road,Heidelberg,Australia
-37.7567527 145.0592739
145 Studley Road,Heidelberg,Australia
-37.7567527 145.0592739
300 Waterdale Road,Heidelberg Heights,Australia
-37.7568089 145.0464521
122 Day St,Bairnsdale,Australia
-37.8313964 147.6077414
33 Cowie Street,Ballan,Australia
-37.59978479999999 144.2223631
1119/1123 Howitt Street,Ballarat,Australia
-37.5408632 143.8321883
272-322 Ryrie Street,Geelong,Australia
-38.1513113 144.3659659
235 Graham Street,Wonthaggi,Australia
-38.6085912 145.581454
141

sh: pause: command not found


32512

In [67]:
# Read in centroid data 
centroid = pd.read_csv('../data/curated/centroid.csv')
centroid = centroid.drop(columns='Unnamed: 0')

,Postcode,Centroid
0,3000,-37.817403 144.956776
1,3001,-37.817403 144.956776
2,3002,-37.818517 144.982207
3,3003,-37.810871 144.949592
4,3004,-37.844246 144.970161
...,...,...
741,8107,-37.844246 144.970161
742,8111,-37.844246 144.970161
743,8120,-37.844246 144.970161
744,8205,-37.844246 144.970161


In [68]:
# Read the hospital csv and merge it with centroid data
hospital = pd.read_csv('../data/raw/hospital_new.csv')
df_merge = pd.merge(centroid, hospital, how='left', on='Postcode')
df_merge = df_merge.dropna()
df_merge.to_csv('1.csv')


In [69]:
from pandas.core.frame import DataFrame

# Create new list to store information
hospital_list = []

for i in range(0,len(df_merge)):
    row = df_merge.iloc[i]
    centroid = row[1]
    hospital = row[5]
    
    # Get longtitude and latitude information by API
    url = 'https://maps.googleapis.com/maps/api/distancematrix/json?origins={}&destinations={}&key={}'.format(centroid,hospital,key)
    payload={}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)

    # Return json
    json_data = json.loads(response.text)
    result = json_data['rows']
    
    # Get distance and duration from the results
    if result[0]['elements'][0]['status'] == 'OK':
        distance = result[0]['elements'][0]['distance']['text']
        duration = result[0]['elements'][0]['duration']['text']
        hospital_list.append([row[0], row[1], row[2], row[3], row[4], row[5], distance, duration])

# Forming new data
hospital_centroid = DataFrame(data = hospital_list, columns=['Postcode', 'Centroid', 'Formal Name', 'Location Address', 'Suburb', 'Latitude and Longitude', 'Distance', 'Duration'])


[{'elements': [{'distance': {'text': '4.0 km', 'value': 4011}, 'duration': {'text': '10 mins', 'value': 580}, 'status': 'OK'}]}]
[{'elements': [{'distance': {'text': '2.0 km', 'value': 1951}, 'duration': {'text': '9 mins', 'value': 563}, 'status': 'OK'}]}]
[{'elements': [{'distance': {'text': '2.9 km', 'value': 2942}, 'duration': {'text': '9 mins', 'value': 526}, 'status': 'OK'}]}]
[{'elements': [{'distance': {'text': '5.2 km', 'value': 5195}, 'duration': {'text': '11 mins', 'value': 650}, 'status': 'OK'}]}]
[{'elements': [{'distance': {'text': '0.6 km', 'value': 645}, 'duration': {'text': '3 mins', 'value': 153}, 'status': 'OK'}]}]
[{'elements': [{'distance': {'text': '1.2 km', 'value': 1218}, 'duration': {'text': '3 mins', 'value': 200}, 'status': 'OK'}]}]
[{'elements': [{'distance': {'text': '1.0 km', 'value': 1021}, 'duration': {'text': '4 mins', 'value': 217}, 'status': 'OK'}]}]
[{'elements': [{'distance': {'text': '0.7 km', 'value': 716}, 'duration': {'text': '3 mins', 'value': 1

In [71]:
# Filter out the closest hospital
hospital_centroid = hospital_centroid.sort_values(by=['Postcode','Distance'], ascending=(True,True))
hospital_centroid.drop_duplicates(subset=['Postcode'], keep='first', inplace=True)

,Postcode,Centroid,Formal Name,Location Address,Suburb,Latitude and Longitude,Distance,Duration
1,3000,-37.817403 144.956776,Melbourne Oral and Facial Surgery Pty Ltd,"Level 12, 63 Exhibition Street",Melbourne,"-37.8144815,144.9714769",2.0 km,9 mins
4,3002,-37.818517 144.982207,East Melbourne Day Procedure Centre,116 - 118 Wellington Parade,East Melbourne,"-37.81621140000001,144.9863185",0.6 km,3 mins
11,3004,-37.844246 144.970161,Alfred Health,Commercial Road,Melbourne,"-37.8458528,144.9863747",3.6 km,8 mins
13,3008,-37.814719 144.948039,Specialist Surgicentre Docklands,"Tenancy 2, 22 Rakaia Way",Docklands,"-37.8146343,144.9413047",0.9 km,3 mins
16,3011,-37.807101 144.907953,Vision Day Surgery Footscray,89 Paisley Street,Footscray,"-37.8006113,144.8950427",1.8 km,5 mins
...,...,...,...,...,...,...,...,...
263,3960,-39.140761 146.360412,South Gippsland Hospital,87 Station Road,Foster,"-38.6588006,146.2061303",75.2 km,1 hour 31 mins
264,3976,-38.043995 145.264296,The Hampton Park Women's Health Care Centre,2 Warana Drive,Hampton Park,"-38.0315896,145.2684897",1.8 km,4 mins
265,3977,-38.1466398 145.2695101,Cranbourne Integrated Care Centre,140-154 Sladen Street,Cranbourne,"-38.1132683,145.2810914",7.3 km,9 mins
266,3981,-38.207298 145.575,Kooweerup Regional Health Service,Rossiter Rd,Kooweerup,"-38.2021198,145.483986",10.0 km,10 mins


In [3]:
# Remove the unit and save into a csv
hospital_centroid['Distance'] = hospital_centroid['Distance'].map(lambda x:x.rstrip('km'))
hospital_centroid.to_csv('../data/curated/closest_hospital.csv')